In [168]:
import pandas as pd
import numpy as np

# Zip Codes Dataset

In [169]:
df = pd.read_csv('data/zip_codes.csv')
state_abbr = pd.read_csv('data/state_abbr.csv')

In [170]:
df.state.unique()

array(['NY', 'PR', 'VI', 'MA', 'RI', 'NH', 'ME', 'VT', 'CT', 'NJ', 'AE',
       'PA', 'DE', 'DC', 'VA', 'MD', 'WV', 'NC', 'SC', 'GA', 'FL', 'AA',
       'AL', 'TN', 'MS', 'KY', 'OH', 'IN', 'MI', 'IA', 'WI', 'MN', 'SD',
       'ND', 'MT', 'IL', 'MO', 'KS', 'NE', 'LA', 'AR', 'OK', 'TX', 'CO',
       'WY', 'ID', 'UT', 'AZ', 'NM', 'NV', 'CA', 'AP', 'HI', 'AS', 'GU',
       'PW', 'FM', 'MP', 'MH', 'OR', 'WA', 'AK'], dtype=object)

In [171]:
df.columns

Index(['zip', 'type', 'decommissioned', 'primary_city', 'acceptable_cities',
       'unacceptable_cities', 'state', 'county', 'timezone', 'area_codes',
       'world_region', 'country', 'latitude', 'longitude',
       'irs_estimated_population'],
      dtype='object')

In [172]:
city_county = df[['primary_city', 'county', 'state']].dropna()
city_county['county'] = city_county['county'].str.title()
city_county.drop_duplicates(inplace=True)
city_county.head()
len(city_county)

29917

In [173]:
# Get full state names, left join on state_abbr
city_county = city_county.merge(state_abbr, left_on='state', right_on='Abbreviation', how='left')
city_county.head()

,primary_city,county,state,State,Abbreviation
0,Holtsville,Suffolk County,NY,New York,NY
1,Adjuntas,Adjuntas Municipio,PR,Puerto Rico,PR
2,Aguada,Aguada Municipio,PR,Puerto Rico,PR
3,Aguadilla,Aguadilla Municipio,PR,Puerto Rico,PR
4,Maricao,Maricao Municipio,PR,Puerto Rico,PR


In [174]:
# rename primary_city to city, drop state
city_county = city_county.drop(columns=['state'])
city_county = city_county.rename(columns={'primary_city': 'city', 'State': 'state', 'Abbreviation': 'abbr'})
city_county['county_state'] = city_county['county'] + ', ' + city_county['state']
city_county.head()

,city,county,state,abbr,county_state
0,Holtsville,Suffolk County,New York,NY,"Suffolk County, New York"
1,Adjuntas,Adjuntas Municipio,Puerto Rico,PR,"Adjuntas Municipio, Puerto Rico"
2,Aguada,Aguada Municipio,Puerto Rico,PR,"Aguada Municipio, Puerto Rico"
3,Aguadilla,Aguadilla Municipio,Puerto Rico,PR,"Aguadilla Municipio, Puerto Rico"
4,Maricao,Maricao Municipio,Puerto Rico,PR,"Maricao Municipio, Puerto Rico"


In [175]:
# state_id, name, abbr
state = city_county[['state', 'abbr']].drop_duplicates().reset_index()
state = state.rename(columns={'state': 'name'})
state_fips = pd.read_csv("data/state-geocodes-v2019.csv")
state = pd.merge(state,state_fips, left_on = "name", right_on = "Name", how="inner")
state = state[['State (FIPS)', 'name', 'abbr']]
state["state_id"] = state["State (FIPS)"]
state = state[['state_id', 'name', 'abbr']]
len(state)

52

In [176]:

county = city_county[['county', 'state']].drop_duplicates().reset_index()
county = county.rename(columns={'county': 'name'})
county = county.merge(state, left_on='state', right_on='name', how='left')
county = county[['state_id', 'name_x', 'state']].rename(columns={'name_x': 'name'})
county['name'] = county['name'].str.title()
county_sort = county.sort_values(by = ['state', 'name'])

def generate_county_index(group):
    # Sort the group alphabetically by county
    sorted_group = group.sort_values(by='name')
    # Generate county index starting from 1 and incrementing by 2
    sorted_group['county_id'] = range(1, len(group) * 2, 2)
    return sorted_group

county_sort = county_sort.groupby('state').apply(generate_county_index).reset_index(drop=True)
county_sort.head(10)
county_sort["state_id"] = county_sort["state_id"].astype(str)
county_sort["county_id"] = county_sort["county_id"].astype(str)

def s_add_zero(s):
    if len(s) == 1:
        return '0' + s
    else:
        return s

def c_add_zero(s):
    if len(s) == 1:
        return '00' + s
    elif len(s) == 2:
        return '0'+s
    else:
        return s
state["state_id"] = state["state_id"].astype(str)
state["state_id"]  = state["state_id"].apply(s_add_zero)
county_sort["state_id"]  = county_sort["state_id"].apply(s_add_zero)
county_sort["county_id"] = county_sort["county_id"].apply(c_add_zero)
county_sort.head()
county_sort.to_csv("data/county_sort1.csv")

## manual edits now need to made to county_sort for discrepancies

In [177]:
# create a city dataframe with
# city_id, state_id, name
county_sort = pd.read_csv("data/county_sort2.csv", dtype = str)
county_sort['county_id'] = county_sort['state_id'] + county_sort['county_id']
county_sort = county_sort[['state_id', 'county_id', 'name', 'state']]
city = city_county[['city', 'county', 'state']].drop_duplicates().reset_index()
city = city.drop(columns=['county'])
city = city.sort_values('city').reset_index()
city['city_id'] = city.index
city = city[['city_id', 'city', 'state']]
city = city.merge(state, left_on='state', right_on='name', how='left')
city = city[['city_id', 'city', 'state_id', 'state']]
city.head()


,city_id,city,state_id,state
0,0,Aaronsburg,42,Pennsylvania
1,1,Abbeville,45,South Carolina
2,2,Abbeville,22,Louisiana
3,3,Abbeville,01,Alabama
4,4,Abbeville,28,Mississippi


In [178]:
# Now make city_county
# city_id, county_id, state_id

city_county = city_county.merge(city, on = ['city', 'state'], how='left')
city_county = city_county.merge(county_sort, left_on=['county', 'state'], right_on=['name', 'state'], how='left')
city_county = city_county[['city_id', 'county_id', 'state_id_x']]
city_county = city_county.rename(columns={'state_id_x': 'state_id'})
city_county = city_county.dropna()
city_county = city_county.drop_duplicates().reset_index()
city_county = city_county.drop(columns=['index'])
city_county.head()

,city_id,county_id,state_id
0,12385,36105,36
1,124,72001,72
2,166,72003,72
3,167,72005,72
4,16337,72101,72


In [179]:
city = city.drop(columns=['state'])
county = county.drop(columns=['state'])

In [180]:
# exports
city.to_csv('data/city.csv', index=False)
city_county.to_csv('data/city_county.csv', index=False)
county_sort.to_csv('data/county.csv', index=False)
state.to_csv('data/state.csv', index=False)